In [1]:
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt
import seaborn as sns
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report, precision_score, recall_score, f1_score
import numpy as np


from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.cluster import KMeans
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report 
from sklearn.metrics import confusion_matrix
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
import numpy as np
import pandas as pd 
import numpy as np
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import precision_score,recall_score,f1_score
from sklearn.metrics import confusion_matrix
from sklearn.svm import SVC


In [2]:
data = pd.read_csv('final_with_covid_AS.csv')
data.head()

,CountCases,BedDys,BeddystoCC,Severity,DiedCases,DiedBdDys,Hsptlsns,April,August,December,...,Y2013,Y2014,Y2015,Y2016,Y2017,Y2018,Y2019,Y2020,Y2021,CovidPrd
0,17,133,7.823529,75,4,9,13,0,0,0,...,1,0,0,0,0,0,0,0,0,0
1,13,372,28.615385,63,4,34,9,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,8,106,13.250000,48,4,13,4,0,0,0,...,0,0,0,0,0,1,0,0,0,0
3,45,218,4.844444,153,3,24,42,0,0,0,...,0,0,0,1,0,0,0,0,0,0
4,24,238,9.916667,90,3,8,21,0,1,0,...,1,0,0,0,0,0,0,0,0,0


In [3]:
filtered_data = data[data['CarDriver'] == 1]
filtered_data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 12582 entries, 11 to 54025
Data columns (total 58 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   CountCases       12582 non-null  int64  
 1   BedDys           12582 non-null  int64  
 2   BeddystoCC       12582 non-null  float64
 3   Severity         12582 non-null  int64  
 4   DiedCases        12582 non-null  int64  
 5   DiedBdDys        12582 non-null  int64  
 6   Hsptlsns         12582 non-null  int64  
 7   April            12582 non-null  int64  
 8   August           12582 non-null  int64  
 9   December         12582 non-null  int64  
 10  February         12582 non-null  int64  
 11  January          12582 non-null  int64  
 12  July             12582 non-null  int64  
 13  June             12582 non-null  int64  
 14  March            12582 non-null  int64  
 15  May              12582 non-null  int64  
 16  November         12582 non-null  int64  
 17  October         

In [4]:
num_bins = 2

bed_days_min = filtered_data['Severity'].min()
bed_days_max = filtered_data['Severity'].max()

bin_width = (bed_days_max - bed_days_min) / num_bins
bins = [bed_days_min + i * bin_width for i in range(num_bins + 1)]

def bin_function(x):
    for i in range(1, len(bins)):
        if x <= bins[i]:
            return i - 1
    return i

filtered_data['Bed_days_category'] = filtered_data['Severity'].apply(bin_function)

/var/folders/3l/xh28kn0j7_7d5hh7fgz188940000gn/T/ipykernel_2153/4218815035.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_data['Bed_days_category'] = filtered_data['Severity'].apply(bin_function)


In [5]:
from xgboost import XGBClassifier

X = filtered_data[['Age1725','Age2639','Age4064','Age6574','Age75plus',
                   
                   'Male','Female',
                   
                   'Regional','MajorCities',

                   'April','December','February','January','July',
                   'June','March','May','November','October','September',

                   'Cntrprt23WldMV','CntrprtFxdStnry','CntrprtHvyVhcl',
                   'CntrprtNClsn','CntrprtOther','CntrprtNMV','CntrprtCrTrkVn',
                   'CntrprtPdlC','CntrprtPdstAnml','CntrprtTrn',

                   'Y2014','Y2015','Y2016','Y2017','Y2018','Y2019','Y2020','Y2021',
                   'Y2013','Y2012',
                   'CovidPrd']]



y = filtered_data['Bed_days_category']


XGBOOST 

In [6]:
from sklearn.model_selection import StratifiedKFold
from sklearn.calibration import CalibratedClassifierCV
from sklearn.metrics import accuracy_score, confusion_matrix, precision_score, recall_score, f1_score
from xgboost import XGBClassifier
import numpy as np

# Define entropy function
def calculate_entropy(probabilities):
    epsilon = 1e-10  # Small constant to avoid log(0)
    return -np.mean(np.sum(probabilities * np.log(probabilities + epsilon), axis=1))

# Number of folds
n_splits = 10

# Initialize StratifiedKFold
skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=42)

# Initialize lists to store metrics for each fold
accuracy_list_before = []
precision_list_before = []
recall_list_before = []
f1_list_before = []
entropy_list_before = []
confusion_matrices_before = []

accuracy_list_after = []
precision_list_after = []
recall_list_after = []
f1_list_after = []
entropy_list_after = []
confusion_matrices_after = []

# Loop through the StratifiedKFold splits
for train_index, test_index in skf.split(X, y):
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]
    
    # Initialize and train the model
    xgb_model = XGBClassifier(random_state=42)
    xgb_model.fit(X_train, y_train)
    
    # Predictions and probabilities before calibration
    y_pred_before = xgb_model.predict(X_test)
    y_prob_before = xgb_model.predict_proba(X_test)
    
    # Calculate entropy before calibration
    entropy_before = calculate_entropy(y_prob_before)
    entropy_list_before.append(entropy_before)
    
    # Store metrics before calibration
    accuracy_list_before.append(accuracy_score(y_test, y_pred_before))
    precision_list_before.append(precision_score(y_test, y_pred_before))
    recall_list_before.append(recall_score(y_test, y_pred_before))
    f1_list_before.append(f1_score(y_test, y_pred_before))
    confusion_matrices_before.append(confusion_matrix(y_test, y_pred_before))
    
    # Calibrate the model
    calibrated_model = CalibratedClassifierCV(xgb_model, method='isotonic', cv=5)
    calibrated_model.fit(X_train, y_train)
    
    # Predictions and probabilities after calibration
    y_pred_after = calibrated_model.predict(X_test)
    y_prob_after = calibrated_model.predict_proba(X_test)
    
    # Calculate entropy after calibration
    entropy_after = calculate_entropy(y_prob_after)
    entropy_list_after.append(entropy_after)
    
    # Store metrics after calibration
    accuracy_list_after.append(accuracy_score(y_test, y_pred_after))
    precision_list_after.append(precision_score(y_test, y_pred_after))
    recall_list_after.append(recall_score(y_test, y_pred_after))
    f1_list_after.append(f1_score(y_test, y_pred_after))
    confusion_matrices_after.append(confusion_matrix(y_test, y_pred_after))
    
    print(f'Confusion Matrix for Fold {len(confusion_matrices_before)} Before Calibration:\n', confusion_matrices_before[-1])
    print(f'Confusion Matrix for Fold {len(confusion_matrices_after)} After Calibration:\n', confusion_matrices_after[-1])

# Calculate mean and standard deviation of each metric before calibration
mean_accuracy_before = np.mean(accuracy_list_before)
std_accuracy_before = np.std(accuracy_list_before)
mean_precision_before = np.mean(precision_list_before)
std_precision_before = np.std(precision_list_before)
mean_recall_before = np.mean(recall_list_before)
std_recall_before = np.std(recall_list_before)
mean_f1_before = np.mean(f1_list_before)
std_f1_before = np.std(f1_list_before)
mean_entropy_before = np.mean(entropy_list_before)
std_entropy_before = np.std(entropy_list_before)

# Calculate mean and standard deviation of each metric after calibration
mean_accuracy_after = np.mean(accuracy_list_after)
std_accuracy_after = np.std(accuracy_list_after)
mean_precision_after = np.mean(precision_list_after)
std_precision_after = np.std(precision_list_after)
mean_recall_after = np.mean(recall_list_after)
std_recall_after = np.std(recall_list_after)
mean_f1_after = np.mean(f1_list_after)
std_f1_after = np.std(f1_list_after)
mean_entropy_after = np.mean(entropy_list_after)
std_entropy_after = np.std(entropy_list_after)

# Calculate mean confusion matrix before and after calibration
mean_conf_matrix_before = np.mean(confusion_matrices_before, axis=0).astype(int)
mean_conf_matrix_after = np.mean(confusion_matrices_after, axis=0).astype(int)

# Print results before calibration
print('--- Before Calibration ---')
print('Mean Accuracy:', mean_accuracy_before)
print('Accuracy Std Dev:', std_accuracy_before)
print('Mean Precision:', mean_precision_before)
print('Precision Std Dev:', std_precision_before)
print('Mean Recall:', mean_recall_before)
print('Recall Std Dev:', std_recall_before)
print('Mean F1-score:', mean_f1_before)
print('F1-score Std Dev:', std_f1_before)
print('Mean Entropy:', mean_entropy_before)
print('Entropy Std Dev:', std_entropy_before)
print('Mean Confusion Matrix:\n', mean_conf_matrix_before)

# Print results after calibration
print('--- After Calibration ---')
print('Mean Accuracy:', mean_accuracy_after)
print('Accuracy Std Dev:', std_accuracy_after)
print('Mean Precision:', mean_precision_after)
print('Precision Std Dev:', std_precision_after)
print('Mean Recall:', mean_recall_after)
print('Recall Std Dev:', std_recall_after)
print('Mean F1-score:', mean_f1_after)
print('F1-score Std Dev:', std_f1_after)
print('Mean Entropy:', mean_entropy_after)
print('Entropy Std Dev:', std_entropy_after)
print('Mean Confusion Matrix:\n', mean_conf_matrix_after)


Confusion Matrix for Fold 1 Before Calibration:
 [[1221    7]
 [   6   25]]
Confusion Matrix for Fold 1 After Calibration:
 [[1208   20]
 [   1   30]]
Confusion Matrix for Fold 2 Before Calibration:
 [[1217   11]
 [   9   22]]
Confusion Matrix for Fold 2 After Calibration:
 [[1206   22]
 [   3   28]]
Confusion Matrix for Fold 3 Before Calibration:
 [[1225    2]
 [  10   21]]
Confusion Matrix for Fold 3 After Calibration:
 [[1213   14]
 [   1   30]]
Confusion Matrix for Fold 4 Before Calibration:
 [[1222    5]
 [   4   27]]
Confusion Matrix for Fold 4 After Calibration:
 [[1210   17]
 [   1   30]]
Confusion Matrix for Fold 5 Before Calibration:
 [[1220    7]
 [   4   27]]
Confusion Matrix for Fold 5 After Calibration:
 [[1213   14]
 [   0   31]]
Confusion Matrix for Fold 6 Before Calibration:
 [[1219    8]
 [   5   26]]
Confusion Matrix for Fold 6 After Calibration:
 [[1201   26]
 [   1   30]]
Confusion Matrix for Fold 7 Before Calibration:
 [[1225    2]
 [   8   23]]
Confusion Matrix f

LOGESTIC REGRESSION

In [7]:
from sklearn.model_selection import StratifiedKFold
from sklearn.calibration import CalibratedClassifierCV
from sklearn.metrics import accuracy_score, confusion_matrix, precision_score, recall_score, f1_score
from sklearn.linear_model import LogisticRegression
import numpy as np

# Define entropy function
def calculate_entropy(probabilities):
    # Add a small constant to avoid log(0)
    epsilon = 1e-10
    return -np.mean(np.sum(probabilities * np.log(probabilities + epsilon), axis=1))

# Number of folds
n_splits = 10

# Initialize StratifiedKFold
skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=42)

# Initialize lists to store metrics for each fold
accuracy_list_before = []
precision_list_before = []
recall_list_before = []
f1_list_before = []
entropy_list_before = []
confusion_matrices_before = []

accuracy_list_after = []
precision_list_after = []
recall_list_after = []
f1_list_after = []
entropy_list_after = []
confusion_matrices_after = []

# Loop through the StratifiedKFold splits
for train_index, test_index in skf.split(X, y):
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]
    
    # Initialize and train the model
    lr_model = LogisticRegression( random_state=42)
    lr_model.fit(X_train, y_train)
    
    # Predict probabilities and classes before calibration
    y_pred_lr_before = lr_model.predict(X_test)
    y_prob_lr_before = lr_model.predict_proba(X_test)
    
    # Calculate metrics and entropy before calibration
    accuracy_list_before.append(accuracy_score(y_test, y_pred_lr_before))
    precision_list_before.append(precision_score(y_test, y_pred_lr_before))
    recall_list_before.append(recall_score(y_test, y_pred_lr_before))
    f1_list_before.append(f1_score(y_test, y_pred_lr_before))
    entropy_list_before.append(calculate_entropy(y_prob_lr_before))
    conf_matrix_before = confusion_matrix(y_test, y_pred_lr_before)
    confusion_matrices_before.append(conf_matrix_before)

    # Calibrate the model
    calibrated_model = CalibratedClassifierCV(lr_model, method='isotonic', cv=5)
    calibrated_model.fit(X_train, y_train)
    
    # Predict probabilities and classes after calibration
    y_pred_lr_after = calibrated_model.predict(X_test)
    y_prob_lr_after = calibrated_model.predict_proba(X_test)
    
    # Calculate metrics and entropy after calibration
    accuracy_list_after.append(accuracy_score(y_test, y_pred_lr_after))
    precision_list_after.append(precision_score(y_test, y_pred_lr_after))
    recall_list_after.append(recall_score(y_test, y_pred_lr_after))
    f1_list_after.append(f1_score(y_test, y_pred_lr_after))
    entropy_list_after.append(calculate_entropy(y_prob_lr_after))
    conf_matrix_after = confusion_matrix(y_test, y_pred_lr_after)
    confusion_matrices_after.append(conf_matrix_after)

# Calculate mean and standard deviation of each metric before calibration
mean_accuracy_before = np.mean(accuracy_list_before)
std_accuracy_before = np.std(accuracy_list_before)
mean_precision_before = np.mean(precision_list_before)
std_precision_before = np.std(precision_list_before)
mean_recall_before = np.mean(recall_list_before)
std_recall_before = np.std(recall_list_before)
mean_f1_before = np.mean(f1_list_before)
std_f1_before = np.std(f1_list_before)
mean_entropy_before = np.mean(entropy_list_before)
std_entropy_before = np.std(entropy_list_before)

# Calculate mean and standard deviation of each metric after calibration
mean_accuracy_after = np.mean(accuracy_list_after)
std_accuracy_after = np.std(accuracy_list_after)
mean_precision_after = np.mean(precision_list_after)
std_precision_after = np.std(precision_list_after)
mean_recall_after = np.mean(recall_list_after)
std_recall_after = np.std(recall_list_after)
mean_f1_after = np.mean(f1_list_after)
std_f1_after = np.std(f1_list_after)
mean_entropy_after = np.mean(entropy_list_after)
std_entropy_after = np.std(entropy_list_after)

# Calculate mean confusion matrix
mean_conf_matrix_before = np.mean(confusion_matrices_before, axis=0)
mean_conf_matrix_after = np.mean(confusion_matrices_after, axis=0)

# Print results before calibration
print('--- Before Calibration ---')
print('Mean Accuracy:', mean_accuracy_before)
print('Accuracy Std Dev:', std_accuracy_before)
print('Mean Precision:', mean_precision_before)
print('Precision Std Dev:', std_precision_before)
print('Mean Recall:', mean_recall_before)
print('Recall Std Dev:', std_recall_before)
print('Mean F1-score:', mean_f1_before)
print('F1-score Std Dev:', std_f1_before)
print('Mean Entropy:', mean_entropy_before)
print('Entropy Std Dev:', std_entropy_before)
print('Mean Confusion Matrix:\n', mean_conf_matrix_before)

# Print results after calibration
print('--- After Calibration ---')
print('Mean Accuracy:', mean_accuracy_after)
print('Accuracy Std Dev:', std_accuracy_after)
print('Mean Precision:', mean_precision_after)
print('Precision Std Dev:', std_precision_after)
print('Mean Recall:', mean_recall_after)
print('Recall Std Dev:', std_recall_after)
print('Mean F1-score:', mean_f1_after)
print('F1-score Std Dev:', std_f1_after)
print('Mean Entropy:', mean_entropy_after)
print('Entropy Std Dev:', std_entropy_after)
print('Mean Confusion Matrix:\n', mean_conf_matrix_after)


--- Before Calibration ---
Mean Accuracy: 0.9915756316050667
Accuracy Std Dev: 0.0015139903351625154
Mean Precision: 0.8717231551348867
Precision Std Dev: 0.06492903048723953
Mean Recall: 0.7760215053763441
Recall Std Dev: 0.05294003876969668
Mean F1-score: 0.8182667982643531
F1-score Std Dev: 0.03205293472670594
Mean Entropy: 0.031937866402866916
Entropy Std Dev: 0.0018317623326474545
Mean Confusion Matrix:
 [[1223.7    3.7]
 [   6.9   23.9]]
--- After Calibration ---
Mean Accuracy: 0.9861711101373766
Accuracy Std Dev: 0.002614662712430351
Mean Precision: 0.6525379657681272
Precision Std Dev: 0.04643640994769721
Mean Recall: 0.9447311827956989
Recall Std Dev: 0.03580887330762542
Mean F1-score: 0.7708867793199803
F1-score Std Dev: 0.03630051050616238
Mean Entropy: 0.02287800149083424
Entropy Std Dev: 0.001381931486583692
Mean Confusion Matrix:
 [[1211.7   15.7]
 [   1.7   29.1]]


Decesion tree

In [19]:
import numpy as np
from sklearn.tree import DecisionTreeClassifier
from sklearn.calibration import CalibratedClassifierCV
from sklearn.metrics import accuracy_score, confusion_matrix, precision_score, recall_score, f1_score
from sklearn.model_selection import StratifiedKFold

# Define entropy function
def calculate_entropy(probabilities):
    # Add a small constant to avoid log(0)
    epsilon = 1e-10
    probabilities = np.clip(probabilities, epsilon, 1 - epsilon)
    entropy = -np.sum(probabilities * np.log(probabilities), axis=1)
    return np.mean(entropy)

# Number of folds
n_splits = 10

# Initialize StratifiedKFold
skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=42)

# Initialize lists to store metrics for each fold
accuracy_list_before = []
precision_list_before = []
recall_list_before = []
f1_list_before = []
entropy_list_before = []
confusion_matrices_before = []

accuracy_list_after = []
precision_list_after = []
recall_list_after = []
f1_list_after = []
entropy_list_after = []
confusion_matrices_after = []

# Loop through the StratifiedKFold splits
for train_index, test_index in skf.split(X, y):
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]
    
    # Initialize and train the model
    dt_model = DecisionTreeClassifier(random_state=42)
    dt_model.fit(X_train, y_train)
    
    # Predict probabilities and classes before calibration
    y_pred_dt_before = dt_model.predict(X_test)
    y_prob_dt_before = dt_model.predict_proba(X_test)
    
    # Calculate metrics and entropy before calibration
    accuracy_list_before.append(accuracy_score(y_test, y_pred_dt_before))
    precision_list_before.append(precision_score(y_test, y_pred_dt_before))
    recall_list_before.append(recall_score(y_test, y_pred_dt_before))
    f1_list_before.append(f1_score(y_test, y_pred_dt_before))
    entropy_list_before.append(calculate_entropy(y_prob_dt_before))
    conf_matrix_before = confusion_matrix(y_test, y_pred_dt_before)
    confusion_matrices_before.append(conf_matrix_before)

    # Calibrate the model
    calibrated_model = CalibratedClassifierCV(dt_model, method='sigmoid', cv=5)
    calibrated_model.fit(X_train, y_train)
    
    # Predict probabilities and classes after calibration
    y_pred_dt_after = calibrated_model.predict(X_test)
    y_prob_dt_after = calibrated_model.predict_proba(X_test)
    
    # Calculate metrics and entropy after calibration
    accuracy_list_after.append(accuracy_score(y_test, y_pred_dt_after))
    precision_list_after.append(precision_score(y_test, y_pred_dt_after))
    recall_list_after.append(recall_score(y_test, y_pred_dt_after))
    f1_list_after.append(f1_score(y_test, y_pred_dt_after))
    entropy_list_after.append(calculate_entropy(y_prob_dt_after))
    conf_matrix_after = confusion_matrix(y_test, y_pred_dt_after)
    confusion_matrices_after.append(conf_matrix_after)

# Calculate mean and standard deviation of each metric before calibration
mean_accuracy_before = np.mean(accuracy_list_before)
std_accuracy_before = np.std(accuracy_list_before)
mean_precision_before = np.mean(precision_list_before)
std_precision_before = np.std(precision_list_before)
mean_recall_before = np.mean(recall_list_before)
std_recall_before = np.std(recall_list_before)
mean_f1_before = np.mean(f1_list_before)
std_f1_before = np.std(f1_list_before)
mean_entropy_before = np.mean(entropy_list_before)
std_entropy_before = np.std(entropy_list_before)

# Calculate mean and standard deviation of each metric after calibration
mean_accuracy_after = np.mean(accuracy_list_after)
std_accuracy_after = np.std(accuracy_list_after)
mean_precision_after = np.mean(precision_list_after)
std_precision_after = np.std(precision_list_after)
mean_recall_after = np.mean(recall_list_after)
std_recall_after = np.std(recall_list_after)
mean_f1_after = np.mean(f1_list_after)
std_f1_after = np.std(f1_list_after)
mean_entropy_after = np.mean(entropy_list_after)
std_entropy_after = np.std(entropy_list_after)

# Calculate mean confusion matrix
mean_conf_matrix_before = np.mean(confusion_matrices_before, axis=0)
mean_conf_matrix_after = np.mean(confusion_matrices_after, axis=0)

# Print results before calibration
print('--- Before Calibration ---')
print('Mean Accuracy:', mean_accuracy_before)
print('Accuracy Std Dev:', std_accuracy_before)
print('Mean Precision:', mean_precision_before)
print('Precision Std Dev:', std_precision_before)
print('Mean Recall:', mean_recall_before)
print('Recall Std Dev:', std_recall_before)
print('Mean F1-score:', mean_f1_before)
print('F1-score Std Dev:', std_f1_before)
print('Mean Entropy:', mean_entropy_before)
print('Entropy Std Dev:', std_entropy_before)
print('Mean Confusion Matrix:\n', mean_conf_matrix_before)

# Print results after calibration
print('--- After Calibration ---')
print('Mean Accuracy:', mean_accuracy_after)
print('Accuracy Std Dev:', std_accuracy_after)
print('Mean Precision:', mean_precision_after)
print('Precision Std Dev:', std_precision_after)
print('Mean Recall:', mean_recall_after)
print('Recall Std Dev:', std_recall_after)
print('Mean F1-score:', mean_f1_after)
print('F1-score Std Dev:', std_f1_after)
print('Mean Entropy:', mean_entropy_after)
print('Entropy Std Dev:', std_entropy_after)
print('Mean Confusion Matrix:\n', mean_conf_matrix_after)


--- Before Calibration ---
Mean Accuracy: 0.9868072296002961
Accuracy Std Dev: 0.0032576254720519454
Mean Precision: 0.7691472144413322
Precision Std Dev: 0.08231432863186067
Mean Recall: 0.6720430107526882
Recall Std Dev: 0.08229284527183661
Mean F1-score: 0.7133129529853914
F1-score Std Dev: 0.06725057592099849
Mean Entropy: 2.4025851012630826e-09
Entropy Std Dev: 0.0
Mean Confusion Matrix:
 [[1220.9    6.5]
 [  10.1   20.7]]
--- After Calibration ---
Mean Accuracy: 0.9870454508145485
Accuracy Std Dev: 0.0026342025692182476
Mean Precision: 0.8103816211456281
Precision Std Dev: 0.1022508168867257
Mean Recall: 0.6331182795698924
Recall Std Dev: 0.06289136916103595
Mean F1-score: 0.7055707913890636
F1-score Std Dev: 0.05285749231145882
Mean Entropy: 0.06283495727999795
Entropy Std Dev: 0.002526201352511382
Mean Confusion Matrix:
 [[1222.4    5. ]
 [  11.3   19.5]]


SVM

In [21]:
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score, confusion_matrix, precision_score, recall_score, f1_score
from sklearn.svm import SVC
from sklearn.calibration import CalibratedClassifierCV
import numpy as np

# Define entropy function
def calculate_entropy(probabilities):
    epsilon = 1e-10
    probabilities = np.clip(probabilities, epsilon, 1 - epsilon)
    entropy = -np.sum(probabilities * np.log(probabilities), axis=1)
    return np.mean(entropy)

# Number of folds
n_splits = 10

# Initialize StratifiedKFold
skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=42)

# Initialize lists to store metrics for each fold
accuracy_list_before = []
precision_list_before = []
recall_list_before = []
f1_list_before = []
entropy_list_before = []
confusion_matrices_before = []

accuracy_list_after = []
precision_list_after = []
recall_list_after = []
f1_list_after = []
entropy_list_after = []
confusion_matrices_after = []

# Loop through the StratifiedKFold splits
for train_index, test_index in skf.split(X, y):
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]
    
    # 1. Fit the base SVM model
    svm_model = SVC(probability=True, random_state=42)
    svm_model.fit(X_train, y_train)
    
    # 2. Predict before calibration
    y_pred_before = svm_model.predict(X_test)
    y_prob_before = svm_model.predict_proba(X_test)
    
    # 3. Calculate entropy before calibration
    entropy_before = calculate_entropy(y_prob_before)
    entropy_list_before.append(entropy_before)
    
    # 4. Store metrics before calibration
    accuracy_list_before.append(accuracy_score(y_test, y_pred_before))
    precision_list_before.append(precision_score(y_test, y_pred_before))
    recall_list_before.append(recall_score(y_test, y_pred_before))
    f1_list_before.append(f1_score(y_test, y_pred_before))
    conf_matrix_before = confusion_matrix(y_test, y_pred_before)
    confusion_matrices_before.append(conf_matrix_before)
    
    # 5. Calibrate the model
    calibrated_model = CalibratedClassifierCV(svm_model, method='isotonic', cv=5)
    calibrated_model.fit(X_train, y_train)
    
    # 6. Predict with the calibrated model
    y_pred_after = calibrated_model.predict(X_test)
    y_prob_after = calibrated_model.predict_proba(X_test)
    
    # 7. Calculate entropy after calibration
    entropy_after = calculate_entropy(y_prob_after)
    entropy_list_after.append(entropy_after)
    
    # 8. Store metrics after calibration
    accuracy_list_after.append(accuracy_score(y_test, y_pred_after))
    precision_list_after.append(precision_score(y_test, y_pred_after))
    recall_list_after.append(recall_score(y_test, y_pred_after))
    f1_list_after.append(f1_score(y_test, y_pred_after))
    conf_matrix_after = confusion_matrix(y_test, y_pred_after)
    confusion_matrices_after.append(conf_matrix_after)

# Calculate mean and standard deviation for each metric before calibration
mean_accuracy_before = np.mean(accuracy_list_before)
std_accuracy_before = np.std(accuracy_list_before)
mean_precision_before = np.mean(precision_list_before)
std_precision_before = np.std(precision_list_before)
mean_recall_before = np.mean(recall_list_before)
std_recall_before = np.std(recall_list_before)
mean_f1_before = np.mean(f1_list_before)
std_f1_before = np.std(f1_list_before)
mean_entropy_before = np.mean(entropy_list_before)
std_entropy_before = np.std(entropy_list_before)
mean_conf_matrix_before = np.mean(confusion_matrices_before, axis=0)

# Calculate mean and standard deviation for each metric after calibration
mean_accuracy_after = np.mean(accuracy_list_after)
std_accuracy_after = np.std(accuracy_list_after)
mean_precision_after = np.mean(precision_list_after)
std_precision_after = np.std(precision_list_after)
mean_recall_after = np.mean(recall_list_after)
std_recall_after = np.std(recall_list_after)
mean_f1_after = np.mean(f1_list_after)
std_f1_after = np.std(f1_list_after)
mean_entropy_after = np.mean(entropy_list_after)
std_entropy_after = np.std(entropy_list_after)
mean_conf_matrix_after = np.mean(confusion_matrices_after, axis=0)

# Print results
print('Before Calibration:')
print('Mean Accuracy:', mean_accuracy_before)
print('Accuracy Std Dev:', std_accuracy_before)
print('Mean Precision:', mean_precision_before)
print('Precision Std Dev:', std_precision_before)
print('Mean Recall:', mean_recall_before)
print('Recall Std Dev:', std_recall_before)
print('Mean F1-score:', mean_f1_before)
print('F1-score Std Dev:', std_f1_before)
print('Mean Entropy:', mean_entropy_before)
print('Entropy Std Dev:', std_entropy_before)
print('Mean Confusion Matrix:\n', mean_conf_matrix_before)

print('\nAfter Calibration:')
print('Mean Accuracy:', mean_accuracy_after)
print('Accuracy Std Dev:', std_accuracy_after)
print('Mean Precision:', mean_precision_after)
print('Precision Std Dev:', std_precision_after)
print('Mean Recall:', mean_recall_after)
print('Recall Std Dev:', std_recall_after)
print('Mean F1-score:', mean_f1_after)
print('F1-score Std Dev:', std_f1_after)
print('Mean Entropy:', mean_entropy_after)
print('Entropy Std Dev:', std_entropy_after)
print('Mean Confusion Matrix:\n', mean_conf_matrix_after)


Before Calibration:
Mean Accuracy: 0.9900654871570163
Accuracy Std Dev: 0.001473869691674251
Mean Precision: 0.7946831797235022
Precision Std Dev: 0.0656481760301978
Mean Recall: 0.8148387096774193
Recall Std Dev: 0.04886455001215023
Mean F1-score: 0.8011321056437085
F1-score Std Dev: 0.02290997196528296
Mean Entropy: 0.028586322636801793
Entropy Std Dev: 0.0020791210136753805
Mean Confusion Matrix:
 [[1220.6    6.8]
 [   5.7   25.1]]

After Calibration:
Mean Accuracy: 0.9876017633294651
Accuracy Std Dev: 0.002278945771890719
Mean Precision: 0.7086686633244462
Precision Std Dev: 0.06811731684069368
Mean Recall: 0.8634408602150536
Recall Std Dev: 0.048623080468172306
Mean F1-score: 0.7746058067465069
F1-score Std Dev: 0.029596336381221833
Mean Entropy: 0.02926872796440213
Entropy Std Dev: 0.0017569548537941476
Mean Confusion Matrix:
 [[1216.    11.4]
 [   4.2   26.6]]


Random Forest

In [17]:
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score, confusion_matrix, precision_score, recall_score, f1_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.calibration import CalibratedClassifierCV
import numpy as np

# Define entropy function
def calculate_entropy(probabilities):
    epsilon = 1e-10
    probabilities = np.clip(probabilities, epsilon, 1 - epsilon)
    entropy = -np.sum(probabilities * np.log(probabilities), axis=1)
    return np.mean(entropy)

# Number of folds
n_splits = 10

# Initialize StratifiedKFold
skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=42)

# Initialize lists to store metrics for each fold
accuracy_list_before = []
precision_list_before = []
recall_list_before = []
f1_list_before = []
entropy_list_before = []
confusion_matrices_before = []

accuracy_list_after = []
precision_list_after = []
recall_list_after = []
f1_list_after = []
entropy_list_after = []
confusion_matrices_after = []

# Loop through the StratifiedKFold splits
for train_index, test_index in skf.split(X, y):
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]
    
    # 1. Fit the base Random Forest model
    rf_model = RandomForestClassifier(random_state=42)
    rf_model.fit(X_train, y_train)
    
    # 2. Predict before calibration
    y_pred_before = rf_model.predict(X_test)
    y_prob_before = rf_model.predict_proba(X_test)
    
    # 3. Calculate entropy before calibration
    entropy_before = calculate_entropy(y_prob_before)
    entropy_list_before.append(entropy_before)
    
    # 4. Store metrics before calibration
    accuracy_list_before.append(accuracy_score(y_test, y_pred_before))
    precision_list_before.append(precision_score(y_test, y_pred_before))
    recall_list_before.append(recall_score(y_test, y_pred_before))
    f1_list_before.append(f1_score(y_test, y_pred_before))
    conf_matrix_before = confusion_matrix(y_test, y_pred_before)
    confusion_matrices_before.append(conf_matrix_before)
    
    # 5. Calibrate the model
    calibrated_model = CalibratedClassifierCV(rf_model, method='isotonic', cv=5)
    calibrated_model.fit(X_train, y_train)
    
    # 6. Predict with the calibrated model
    y_pred_after = calibrated_model.predict(X_test)
    y_prob_after = calibrated_model.predict_proba(X_test)
    
    # 7. Calculate entropy after calibration
    entropy_after = calculate_entropy(y_prob_after)
    entropy_list_after.append(entropy_after)
    
    # 8. Store metrics after calibration
    accuracy_list_after.append(accuracy_score(y_test, y_pred_after))
    precision_list_after.append(precision_score(y_test, y_pred_after))
    recall_list_after.append(recall_score(y_test, y_pred_after))
    f1_list_after.append(f1_score(y_test, y_pred_after))
    conf_matrix_after = confusion_matrix(y_test, y_pred_after)
    confusion_matrices_after.append(conf_matrix_after)

# Calculate mean and standard deviation for each metric before calibration
mean_accuracy_before = np.mean(accuracy_list_before)
std_accuracy_before = np.std(accuracy_list_before)
mean_precision_before = np.mean(precision_list_before)
std_precision_before = np.std(precision_list_before)
mean_recall_before = np.mean(recall_list_before)
std_recall_before = np.std(recall_list_before)
mean_f1_before = np.mean(f1_list_before)
std_f1_before = np.std(f1_list_before)
mean_entropy_before = np.mean(entropy_list_before)
std_entropy_before = np.std(entropy_list_before)
mean_conf_matrix_before = np.mean(confusion_matrices_before, axis=0)

# Calculate mean and standard deviation for each metric after calibration
mean_accuracy_after = np.mean(accuracy_list_after)
std_accuracy_after = np.std(accuracy_list_after)
mean_precision_after = np.mean(precision_list_after)
std_precision_after = np.std(precision_list_after)
mean_recall_after = np.mean(recall_list_after)
std_recall_after = np.std(recall_list_after)
mean_f1_after = np.mean(f1_list_after)
std_f1_after = np.std(f1_list_after)
mean_entropy_after = np.mean(entropy_list_after)
std_entropy_after = np.std(entropy_list_after)
mean_conf_matrix_after = np.mean(confusion_matrices_after, axis=0)

# Print results
print('Before Calibration:')
print('Mean Accuracy:', mean_accuracy_before)
print('Accuracy Std Dev:', std_accuracy_before)
print('Mean Precision:', mean_precision_before)
print('Precision Std Dev:', std_precision_before)
print('Mean Recall:', mean_recall_before)
print('Recall Std Dev:', std_recall_before)
print('Mean F1-score:', mean_f1_before)
print('F1-score Std Dev:', std_f1_before)
print('Mean Entropy:', mean_entropy_before)
print('Entropy Std Dev:', std_entropy_before)
print('Mean Confusion Matrix:\n', mean_conf_matrix_before)

print('\nAfter Calibration:')
print('Mean Accuracy:', mean_accuracy_after)
print('Accuracy Std Dev:', std_accuracy_after)
print('Mean Precision:', mean_precision_after)
print('Precision Std Dev:', std_precision_after)
print('Mean Recall:', mean_recall_after)
print('Recall Std Dev:', std_recall_after)
print('Mean F1-score:', mean_f1_after)
print('F1-score Std Dev:', std_f1_after)
print('Mean Entropy:', mean_entropy_after)
print('Entropy Std Dev:', std_entropy_after)
print('Mean Confusion Matrix:\n', mean_conf_matrix_after)


Before Calibration:
Mean Accuracy: 0.9895886658980617
Accuracy Std Dev: 0.002172952144176431
Mean Precision: 0.8500099369021783
Precision Std Dev: 0.07014333652778787
Mean Recall: 0.704731182795699
Recall Std Dev: 0.06449677128945128
Mean F1-score: 0.7677281115755861
F1-score Std Dev: 0.04870180194488136
Mean Entropy: 0.029155981591317142
Entropy Std Dev: 0.0022529430905308206
Mean Confusion Matrix:
 [[1223.4    4. ]
 [   9.1   21.7]]

After Calibration:
Mean Accuracy: 0.9840252250568561
Accuracy Std Dev: 0.0030643626662571837
Mean Precision: 0.6284102102924691
Precision Std Dev: 0.05461064173289607
Mean Recall: 0.8763440860215054
Recall Std Dev: 0.041536564136309316
Mean F1-score: 0.7301312546703954
F1-score Std Dev: 0.03968642397492199
Mean Entropy: 0.031426191101187315
Entropy Std Dev: 0.0027095447139148885
Mean Confusion Matrix:
 [[1211.1   16.3]
 [   3.8   27. ]]
